# 9장. 문서 요약 (Text Summarization)

# 9-0 데이터 준비

In [67]:
import requests 
from bs4 import BeautifulSoup
import re

def get_news_by_url(url):
    headers = {"user-agent" : "Mozilla/5.0"}
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, 'html.parser')
    text = re.sub("\s*.*오류를 우회하기 위한 함수 추가\s.*\s+","",soup.select_one("#articleBodyContents").text)
    text = re.sub(".{3} \(\w+@.+\).+\s+","",text)
    text = re.sub("[^가-힣A-z0-9\s\.\?\!]+"," ",text)
    text = re.sub("\.+",". ",text)
    text = re.sub("\[.+\]", " ",text)
    text = re.sub("\s+"," ",text).strip()
    
    return text
    
doc = get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=018&aid=0004430108')
doc[:100]

'과기정통부 22일 유영민 장관 등 참석해 기념행사2021년까지 1516억원 투입 5100여종 데이터 구축민간 클라우드 통한 외부연계체계도. 개방성 강화 국가 차원의 빅데이터 활용 '

## Mecab 설치 (필요시)

!sudo apt-get install g++ openjdk-7-jdk # Install Java 1.7+
!sudo apt-get install python-dev; pip install konlpy     # Python 2.x
!sudo apt-get install python3-dev; pip3 install konlpy   # Python 3.x
!sudo apt-get install curl
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

# 9-1 Luhn Summarizer

http://courses.ischool.berkeley.edu/i256/f06/papers/luhn58.pdf

Hans Peter Luhn

https://en.wikipedia.org/wiki/Hans_Peter_Luhn

 Hans Peter Luhn은 공학과 정보과학에서의 개척 작업으로 "정보 검색의 아버지"로 알려져 있다. 그는 표제어가 문맥에 포함된 채 배열된 색인(KWIC : keyword-in-context) 개발, 정보 선택 제공(SDI), 완전 텍스트 프로세싱, 자동 발췌(요약), 단어 시소로스의 최초 현대식 사용으로 신뢰를 얻었다. 오늘날 파생된 지식 대부분에는 KWIC 색인이 있으며 과학의 모든 분야에 SDI시스템이 있다. 


 Luhn은 1896년 7월 1일 독일 바르멘에서 태어났다. 아버지가 그 당시 유명한 인쇄업자였으므로, 스위스에서 인쇄업을 배웠다. 어렸을 때무터 그는 창조성이 뛰어난 재능을 보였으며, 기술적 문제, 물리학, 통계학에 관심을 보였다. 그러나 1차 세계대전으로 독일군 통신장교로 복역(1915년1917년)하면서 프랑스 터키 루마니아 불가리아 등지로 옮겨 다녀야만 했다. 1차 대전 이후 그는 스위스의 Gallen 교회 Schweizwrische handels Hochschule로 돌아와 기술, 물리학, 회계분야의 수업을 들었다. 그 후, Luhn은 그리스에서 못다한 공부에 전념했으며, 더블부기기계(Duble-Entry Bookeeping Machine : 카드 대장에 대변기입과 차변 기입을 기록할 수 있는 것)를 발명하였다. 그는 또 Hollerith tabulating/recording machine에 정통했고, 천공카드에서 문자 숫자를 나타내는 장치의 사용을 증가시키게 했다. 1920년부터 1930년까지는 10개의 특허권을 획득하여 그의 탁월한 능력을 보여주었다. 그것들중에 루노메터(Lunometer : 직물의 실길이를 계산하는데 쓰이는 장치)는 지금도 사용되고 있다.


 1920년까지 그는 직물 공장에서 일하기 시작했다. 그는 직물 공장의 사업확장을 위해 뉴잉글랜드에서 1924년 미국으로 가게 되었다. 그러나 회사가 곧 파산하였고 Luhn은 직장없이 뉴욕에 남게 되었다. 그는 은행에서 일을 하였고 곧 뉴욕 월스트리트에 소재한 국제어음은행(International Acceptance Bank)에서 재정담당관으로 승진하였다. 


 1933년 Luhn은 자사인 공학회사 H.P. Luhn & Association을 설립하였고 8년간 자문기술자로 일했다. 1941년 Luhn은 IBM에서 수석 연구기술자로 참여하였고 이후에 정보검색연구 관리자로 일했다. Luhn이 IBM에서 새로운 아이디어를 지속적으로 내놓고 문제를 다르게 접근하여 주목을 받는 동안, 다른 기술자들에게 고차원적인 창조를 하도록 자극하면서 그들의 촉매제 역할을 하여 신뢰를 얻었다

In [30]:
doc

'과기정통부 22일 유영민 장관 등 참석해 기념행사2021년까지 1516억원 투입 5100여종 데이터 구축민간 클라우드 통한 외부연계체계도. 개방성 강화 국가 차원의 빅데이터 활용 시대가 열린다. 새로운 산업 창출과 기존 산업의 변화에 이르는 혁신성장 을 위한 센터가 문을 연다. 10개 분야에 걸쳐 데이터 경제 의 발전을 위한 정부의 청사진을 현실로 구현하는데 앞장선다는 계획이다. 22일 과학기술정보통신부는 서울 중구 대한상공회의소에서 데이터 생태계 조성과 혁신 성장의 기반 마련을 위한 빅데이터 플랫폼 및 센터 출범식 행사를 개최했다. 유영민 과기정통부 장관을 비롯해 노웅래 국회 과학기술정보방송통신위원회 위원장 등 300여명이 참가했다. 10개 분야 100개 센터. 3년간 1516억원 투입이미지 픽사베이빅데이터는 데이터 활용을 통해 혁신성장을 이루자는 문재인 정부의 경제 성장 핵심 요소중 하나다. 문재인 대통령이 직접 올 들어 데이터 활용과 이에 따른 정보보호 보안 에 대한 중요성을 강조하기도 했다. 이런 맥락 속에서 빅데이터센터는 공공과 민간이 협업해 활용도 높은 양질의 데이터를 생산 구축하고 플랫폼은 이를 수집 분석 유통하는 역할을 담당한다. 과기정통부는 분야별 플랫폼 10개소와 이와 연계된 기관별 센터 100개소를 구축하는데 3년간 총 1516억원을 투입할 계획이며 올해 우선 640억원 규모의 사업을 추진하고 있다. 대상 분야는 금융 BC카드 환경 한국수자원공사 문화 한국문화정보원 교통 한국교통연구원 헬스케어 국립암센터 유통 소비 매일방송 통신 KT 중소기업 더존비즈온 지역경제 경기도청 산림 한국임업진흥원 등으로 현재 1차 공모를 통해 72개 빅데이터 센터를 선정했고 다음달 8일까지 2차 공모를 통해 28개를 추가 선정해 총 100개를 지원 운영할 계획이다. 이를 통해 데이터 생태계를 혁신하고 기업의 경쟁력을 제고하는 역할을 수행한다. 주요 활용 전략 사례를 보면 빅데이터 활용을 통해 신 시장 을 창출하는 방안을 담고 있다. 금융 플랫폼의 경우 소상공인 

In [4]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/ryleyun/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### 1) 토큰화

In [34]:
from nltk.tokenize import sent_tokenize
from konlpy.tag import Mecab
mecab = Mecab()

# 문장 분리
def get_sentences(txt):     
    return sent_tokenize(txt)

# 토큰화 
def get_words(txt):
    return [word for word, pos in mecab.pos(txt) if pos[0] == 'N' and len(word)>1]

### 2) 중요 단어 결정

In [36]:
# 단어(토큰)의 가중치 계산 및 범위에 포함되는 토큰 식별 
def get_keywords(word_list , min_ratio=0.001, max_ratio=0.5) :
    # assert는 방어적 프로그래밍 방법으로, true면 넘어가고 아니면 error 발생시킴.
    assert (min_ratio < 1 and max_ratio < 1)

    # 토큰별로 빈도수 카운팅
    count_dict = {}
    for word in word_list:
#         if word in count_dict.keys():
#             count_dict[word] += 1
#         else:
#             count_dict[word] = 1
        
    # setdefault를 통해 하나 추가하는데, 만약 기존에 key가 있으면 그냥 넘어감
        count_dict.setdefault(word, 0)
        count_dict[word] += 1
        
    # 분석 문서의 총 토큰수 대비 해당 토큰의 빈도 비율   
    keywords = set()
    for word, cnt in count_dict.items():
        word_percentage = cnt/len(word_list)
        
        # 사전 정의한 비율내에 포함 된 경우 키워드에 추가        
        if word_percentage >= min_ratio and word_percentage <= max_ratio:
            keywords.add(word)

    return keywords

get_keywords(['바나나','사과','바나나','바나나','포도'])

{'사과', '포도'}

### 3) 문장 중요도 계산

In [44]:
# 문장의 가중치 계산
def get_sentence_weight (sentence , keywords):
    tokens = sentence.split(" ")
    window_start, window_end = 0, -1
    
    # 문장내에서 윈도 시작 위치 탐색
    for i in range(len(tokens)):
        # 범위내 속한 키워드가 등장하는 첫번째 위치 계산
        if tokens[i] in keywords:
            window_start = i
            break
    
    # 문장내에서 윈도 종료 위치 탐색
    for i in range(len(tokens) - 1, 0, -1):
        # 범위내 속한 키워드가 등장하는 마지막 위치 계산
        if tokens[i] in keywords:
            window_end = i 
            break
        
    
    # 윈도의 시작위치가 종료위치보다 큰경우 => 분석할 단어(토큰)가 없는 경우 종료
    if window_start > window_end:
        return 0
        
    
    # 윈도 크기 계산
    window_size = window_end - window_start + 1
    
    
    # 분석 대상 문장 중 범위(0.001 ~ 0.5)에 포함된 토큰 개수 카운팅
    keyword_cnt = 0
    for w in tokens[window_start : window_start + window_size]:
        if w in keywords:
            keyword_cnt += 1
    
    
    # (분석 대상 문장 중 범위(0.001 ~ 0.5)에 포함된 토큰 개수)의 제곱 / 윈도사이즈
    return keyword_cnt * keyword_cnt / float(window_size)

### 4) 문서 요약

In [53]:
# 문서 요약
def summarize(content, max_no_of_sentences = 10):
    
    # 단어(토큰) 분리
    word_list = get_words(content)
    
    # 단어(토큰) 가중치 계산 및 범위 내 포함 단어(토큰) 추출
    keywords = get_keywords(word_list)
    
    # 문장별 가중치 계산
    sentence_list = get_sentences(content)
    sentence_weight = []
    
    for sen in sentence_list:
        sentence_weight.append((get_sentence_weight(sen, keywords), sen))
           
    # 문장별 가중치 역순 계산
    sentence_weight.sort(reverse=True)
#     print(sentence_weight)
    
    return [weight[1] for weight in sentence_weight[:max_no_of_sentences]]

In [60]:
li = summarize (doc ,  3)
for i, s in enumerate(li) :
    print(f"{i+1})", s)
    print("\n")

1) 의료비 절감 헬스케어 과 기업의 매출 향상을 통한 산업 육성 통신 산림 등도 눈길을 끈다.


2) 22일 과학기술정보통신부는 서울 중구 대한상공회의소에서 데이터 생태계 조성과 혁신 성장의 기반 마련을 위한 빅데이터 플랫폼 및 센터 출범식 행사를 개최했다.


3) 대상 분야는 금융 BC카드 환경 한국수자원공사 문화 한국문화정보원 교통 한국교통연구원 헬스케어 국립암센터 유통 소비 매일방송 통신 KT 중소기업 더존비즈온 지역경제 경기도청 산림 한국임업진흥원 등으로 현재 1차 공모를 통해 72개 빅데이터 센터를 선정했고 다음달 8일까지 2차 공모를 통해 28개를 추가 선정해 총 100개를 지원 운영할 계획이다.






---



# 9-2 Textrank

![대체 텍스트](https://www.researchgate.net/profile/Khushboo_Thakkar3/publication/232645575/figure/fig1/AS:575720050573312@1514273764062/Sample-graph-build-for-sentence-extraction.png)

## 2.1 TextRank 직접 구현하기 (Matrix 활용)

### 1) 자카드 유사도

In [23]:
Text = "딸기 바나나 사과 파인애플. 바나나 사과 딸기 포도. 복숭아 수박. 파인애플 사과 딸기 바나나."

In [24]:
from konlpy.tag import Mecab
mecab = Mecab()
# 문장간 유사도 측정 (자카드 유사도 사용)
def sentence_similarity(sentence1, sentence2):
    sentence1 = [token[0] for token in mecab.pos(sentence1) if token[1][0] in ["N","V"]]
    sentence2 = [token[0] for token in mecab.pos(sentence2) if token[1][0] in ["N","V"]]
    union = set(sentence1) | set(sentence2)
    intersection = set(sentence1) & set(sentence2)
    
    return len(intersection)/len(union)

sentence_similarity('나는 치킨을 좋아해','나는 치킨을 싫어해')

0.5

### 2) 그래프 생성

In [61]:
import numpy as np
from nltk.tokenize import sent_tokenize

def buildMatrix(sentences):
    score = np.ones(len(sentences), dtype=np.float32)
    weighted_edge = np.zeros((len(sentences),len(sentences)), dtype=np.float32)
    
    # 문장별로 그래프 edge를 Matrix 형태로 생성
    for i in range(len(sentences)):
        for j in range(len(sentences)):
            if i == j:
                continue
            weighted_edge[i][j] = sentence_similarity(sentences[i], sentences[j])

    # normalize 
    for i in range(len(weighted_edge)):
        score[i] = weighted_edge[i].sum()
        weighted_edge[i] /= score[i]
     
    return weighted_edge

Text = "딸기 바나나 사과 파인애플 수박. 바나나 사과 딸기 포도. 복숭아 수박. 파인애플 사과 딸기 바나나."
buildMatrix(sent_tokenize(Text))    

array([[0.        , 0.3380282 , 0.1690141 , 0.49295774],
       [0.41379312, 0.        , 0.10344828, 0.48275864],
       [0.5       , 0.25      , 0.        , 0.25      ],
       [0.5072464 , 0.4057971 , 0.08695652, 0.        ]], dtype=float32)

### 3) 문장 중요도 계산

In [62]:
def scoring(weighted_edge, score, threshold=0.0001, d=0.85, max_iter = 50):
    for iter in range(max_iter):
        new_score = (1-d) * d * weighted_edge.T.dot(score)
        if abs(new_score - score).sum() <= threshold:
            break
            
        score = new_score
        
    return new_score

### 4) 문서 요약

In [63]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/ryleyun/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [64]:
from nltk.tokenize import sent_tokenize

def summarize(text, n=10):
    sentences = sent_tokenize(text)
    
    weighted_edge = buildMatrix(sentences)
    init_score = np.ones(len(sentences), dtype=np.float32)
    score = scoring(weighted_edge, init_score)
    
    sorted_score = sorted(enumerate(score), key = lambda x: x[1], reverse=True)[:n]
    
    return [sentences[idx] for idx, sent in sorted_score]

In [65]:
summary = summarize(Text, 3)

for sent in summary :
    print(sent)

딸기 바나나 사과 파인애플 수박.
파인애플 사과 딸기 바나나.
바나나 사과 딸기 포도.




---



## 2.2 TextRank 직접 구현하기 (Graph 활용)

In [47]:
Text = "딸기 바나나 사과 딸기 파인애플. 바나나 사과 딸기. 복숭아 파인애플. 파인애플 사과 딸기 바나나."

### 2) 토큰화

In [48]:
import nltk
# nltk.download('punkt')
from nltk.tokenize import sent_tokenize

In [49]:
def sentences(text):
    return sent_tokenize(text)

### 3) 자카드 유사도

In [50]:
from konlpy.tag import Mecab

# 문장간 유사도 측정 (자카드 유사도 사용)
def sentence_similarity(sentence1, sentence2):
    mecab = Mecab()
    sent1 = mecab.morphs(sentence1)
    sent2 = mecab.morphs(sentence2)

    return len(set(sent1)&set(sent2))/len(set(sent1)|set(sent2))

sentence_similarity('나는 치킨을 좋아해','나는 치킨을 싫어해')

0.6666666666666666

In [55]:
def connect(nodes):
    return [(start, end, sentence_similarity(start, end)) for start in nodes for end in nodes if start is not end]

### 3) 그래프 생성

In [56]:
import networkx as nx

def rank(nodes,edges):
    graph=nx.diamond_graph()
    graph.clear() 
    graph.add_nodes_from(nodes)
    graph.add_weighted_edges_from(edges)

    return nx.pagerank(graph)

### 4) 문서 요약

In [59]:
def summarize(text,num_summaries=3):
    nodes = sentences(text)
    edges = connect(nodes)
    scores = rank(nodes, edges)
    
    return sorted(scores.items(), key = lambda x: x[1], reverse=True)[:num_summaries]

In [60]:
summary=summarize(Text, 3)

for sent in summary :
  print(sent)

('딸기 바나나 사과 딸기 파인애플.', 0.3034803909925603)
('파인애플 사과 딸기 바나나.', 0.3034803909925603)
('바나나 사과 딸기.', 0.254517723082264)


## 2.3 gensim을 활용한 요약

In [68]:
from gensim.summarization.summarizer import summarize
summarize(doc).split('\n')

['대상 분야는 금융 BC카드 환경 한국수자원공사 문화 한국문화정보원 교통 한국교통연구원 헬스케어 국립암센터 유통 소비 매일방송 통신 KT 중소기업 더존비즈온 지역경제 경기도청 산림 한국임업진흥원 등으로 현재 1차 공모를 통해 72개 빅데이터 센터를 선정했고 다음달 8일까지 2차 공모를 통해 28개를 추가 선정해 총 100개를 지원 운영할 계획이다.',
 '이를 통해 데이터 생태계를 혁신하고 기업의 경쟁력을 제고하는 역할을 수행한다.',
 'AI 알고리즘 제공도센터는 우선 분야별 데이터 부족 문제를 해소하기 위해 올해 말까지 시장 수요가 높은 1400여종 신규 데이터를 생산 구축하고 사업이 완료되는 2021년까지 총 5100여종 양질의 풍부한 데이터를 생산 구축해 시장에 공급할 계획이다.',
 '100개 센터에서 수집된 데이터를 융합 분석한 뒤 맞춤형 데이터 제작 등 양질의 데이터로 재생산하고 기업들이 필요로 하는 데이터를 원하는 형태로 즉시 활용할 수 있도록 제공할 계획이다.',
 '유영민 과기정통부 장관은 오늘 출범식은 대한민국이 데이터 강국으로 가기 위한 초석을 놓은 자리 라며 세계 주요국들보다 데이터 경제로 나아가는 발걸음이 다소 늦었지만 빅데이터 플랫폼과 센터를 지렛대로 우리나라의 낙후된 데이터 생태계를 혁신하고 기업의 경쟁력을 한 단계 제고할 수 있도록 정책적 역량을 집중하겠다 고 밝혔다.']



---

